In [1]:
%matplotlib inline

In [77]:
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
pd.options.mode.chained_assignment = None #default = 'warn'

sns.set(style="whitegrid")

# You may load the dataset from URL (instead of the local file) if you wish
auto_mpg_data = "https://raw.githubusercontent.com/fundamentals-of-data-science/datasets/master/auto-mpg.data.txt"
auto_mpg_names = "https://raw.githubusercontent.com/fundamentals-of-data-science/datasets/master/auto-mpg.names.txt"

# Problem Set 5.1

This Problem Set is a two parter. In the first part you will apply ETL/EDA/Mathematical Modeling (Labs 5 and 6) to a new data set.
Each part is a separate grade.
Take all your PS3 comments into account.

## Directions

1. Show all work/steps/calculations using a combination of code and Markdown.
2. **All** work is to be your own. This is not a group project. You may, however, use code from the lectures and labs. Provide citations for any code that is not your own. You may also consult Stackoverflow, etc. This is not by any means "closed book" or anything like that. Basically, I need to see if *you* learned the concepts from the Lectures, *Fundamentals*, and Labs.
3. Add whatever markdown or code cells you need in each part to explain and calculate your answers. Don't just provide answers but explain them as well. **Explain and interpret your results.**


<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Note</strong>
    <p>This part of the Problem Set covers Labs 5 and 6. You get even less help this time. You should just know what to do.</p>
    <p>Additionally, this is a <em>report</em>, a narrative description of your process and findings. Use full sentences. Limit bullet lists. You should be able to "hide code" and have the remaining text make sense.</p></div>

## 1.1 - ETL

Let's first start by taking a look at the data and names files we've read in. Since these are text files, I'm going to manually take a quick peek at these files.

From our names file, we have our attribute information for our data.

```
Description of fields in auto mpg data.

Name            Data Type
----            ---------
mpg             continuous
cylinders       multi-valued discrete
displacement    continuous
horsepower      continuous
weight          continuous
acceleration    continuous
model year      multi-valued discrete
origin          multi-valued discrete
car name        string (unique for each instance)
```

So we have 9 different variables in this dataset (most appear to be numeric). We know from our names file that there should be 398 instances/observations, and the $horsepower$ variable has 6 missing values.

We will cover these in more detail during the EDA process. For now let's try reading in the data. 

In [23]:
auto_mpg_data

'https://raw.githubusercontent.com/fundamentals-of-data-science/datasets/master/auto-mpg.data.txt'

This is indeed a text file with some data in it. Our first step should be getting our data into tidy data. Since there is an inconsistent amount of whitespace between each entry for any particular observation, we can tell our `pd.read_csv()` method to consider any whitespace between a value as a separator. (I initially used the `delim_whitespace=True` parameter, though that seems to be depreciated in favor of `\s+` now).

Since there are no headers in this text file, I will manually add the columns to our dataframe, using the description that we found above in our names file.

In [40]:
columns = [
    'mpg', 
    'cylinders', 
    'displacement', 
    'horsepower',
    'weight', 
    'acceleration', 
    'model year',
    'origin', 
    'car name'
]

data = pd.read_csv(auto_mpg_data, sep='\s+', header=None, names=columns)

We'll check the first few entries to make sure our data has been read in properly.

In [41]:
data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


Looks like we got most of what we need. We have 9 columns to represent each feature/attribute. We can also look at the information provided here with the `info()` method.

In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    float64
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   car name      398 non-null    object 
dtypes: float64(4), int64(3), object(2)
memory usage: 28.1+ KB


We do in fact see 398 rows for each attribute. Most variables are floats or integers, with the exceptions of $horsepower$ and $car name$, the latter which we expect to be strings, so this is not unexpected. But we know $horsepower$ has 6 missing values somewhere. How should we handle this? I'm going to simply look at our observations first, where $horsepower$ does not have a value (from peeking at the data text file, I know the missing values are coded as a '?').

In [43]:
data[data['horsepower'] == '?']

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
32,25.0,4,98.0,?,2046.0,19.0,71,1,ford pinto
126,21.0,6,200.0,?,2875.0,17.0,74,1,ford maverick
330,40.9,4,85.0,?,1835.0,17.3,80,2,renault lecar deluxe
336,23.6,4,140.0,?,2905.0,14.3,80,1,ford mustang cobra
354,34.5,4,100.0,?,2320.0,15.8,81,2,renault 18i
374,23.0,4,151.0,?,3035.0,20.5,82,1,amc concord dl


We do in fact only have 6 observations here. Because that's only (6/398) = 0.015, or about 1.5% of our data, I will consider imputation for these 6 values with the median of $horsepower$. This should not create inliers or other data abnormalities such as loss of variation since there are only a small number of missing values. We might turn to some domain knowledge to think about whether our median is a good estimate for these missing values.

[Average Horsepower of a Car Over the Years](https://carbuzz.com/features/average-horsepower-of-a-car-over-the-years/)

In this article, we get different ranges for the average horsepower of cars in the United States, mostly broken down by decade. To summarize, the 50s started out with an average of about 100 horsepower, and only went up to about 120hp by the 80s. Later decades saw a bit of increase, where the average could be considered closer to 175hp or so, but this might at least give us an idea if we're on the mark with imputing our missing values.

I know this is kind of blending into the EDA portion a bit, so I will just replace the missing values with our median here, and as long as we have our tidy data, we can start the EDA process and go more in depth with horsepower in the next section.

We need to select all the other 392 rows that don't include the 6 missing values in order to get the median for $horsepower$.

In [73]:
data_no_missing = data[data['horsepower'] != '?']

Since the $horsepower$ variable was a mix of strings and floats, it caused all the values to become strings. We fix this below so we can perform the numpy operations on this column.

In [75]:
data_no_missing['horsepower'] = data_no_missing['horsepower'].astype('float64')

C:\Users\Jhirs\AppData\Local\Temp\ipykernel_25224\1856811709.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_no_missing['horsepower'] = data_no_missing['horsepower'].astype('float64')


Here we can finally find the median of $horsepower$ without the missing values. 

In [76]:
np.median(data_no_missing['horsepower'])

93.5

It's 93.5, which seems like a reasonable guess for our missing values, if not slightly on the low side, remembering that we theorized average car horsepowers range between 100-200, very roughly. (Again, we could also look at the median model years for the cars to get a better rough estimate, but I think that's getting too much into the EDA portion of things).

Finally, we can replace our missing values with our median of 93.5 and convert the column into a float type to prepare for our EDA.

In [80]:
data = data.replace('?', '93.5')
data['horsepower'] = data['horsepower'].astype('float64')

In [81]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    float64
 4   weight        398 non-null    float64
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   car name      398 non-null    object 
dtypes: float64(5), int64(3), object(1)
memory usage: 28.1+ KB


We can see that our $horsepower$ variable is now a float64 type instead of object like it was originally. We are ready to move on to EDA now.

## 1.2 - EDA


## 1.3 - Null & Distributional Models

---

**PRE-SUBMISSION CHECK LIST**

Before you submit this assignent, go back and review the directions to ensure that you have followed each instruction.

* [ ] Have you completed every section and answered every question asked?
* [ ] For every question, have you described your approach and explained your results?
* [ ] Have you checked for spelling and grammar errors?
* [ ] Are your code blocks free of any errors?
* [ ] Have you deleted unused code or markdown blocks? Removed scratch calculations? Excessive raw data print outs?
* [ ] Hide all the code/output cells and make sure that you have sufficient discussion. Re-show the output cells but leave code cells hidden.
* [ ] Have you *SAVED* your notebook?
* [ ] Are you following the submission requirements for this particular assignment?